In [ ]:
import requests
import pandas as pd
import random
from google.colab import drive
import csv
from google.colab import auth
import gspread
from google.auth import default
import json

auth.authenticate_user()
drive.mount("/drive")
creds, _ = default()
gc = gspread.authorize(creds)
overpass_url = "http://overpass-api.de/api/interpreter"

Mounted at /drive


In [ ]:
worksheetname = 'short_name_templates'
data_length = [10,50,100,250,500, 1000]

In [ ]:
templates = []
templatesheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/17xRVmIc5XNd4IPAlzcFleWM6QRLtEJsAWEqp-TY4sEU/edit#gid=1233379409')
sheet = templatesheet.worksheet(worksheetname)
templates = sheet.get_all_values()
firstrow = templates[0]
templates.pop(0)

['Text', 'FIRSTNAME', 'MIDDLENAME', 'LASTNAME']

In [ ]:
def get_random_namerow():
  n = random.randint(0,len(templates)-1)
  return templates[n]

In [ ]:
def grab_name_data():
  data_retrieved = False
  while (data_retrieved == False):
    try:
      data = json.loads(requests.get('https://randomuser.me/api/?nat=us,de,gb&inc=name&results=2').content.decode('utf-8'))
      return {
          'first': data['results'][0]['name']['first'],
          'middle': data['results'][1]['name']['first'],
          'last': data['results'][0]['name']['last']
      }
    except:
      data_retrieved = False

In [ ]:
def get_value_of_tag(data, tag):
  a_dictionary = data
  if tag not in a_dictionary:
    return ""
  else:
    return a_dictionary[tag].replace(".", "").replace("(", "").replace(")", "")

In [ ]:
def combine_data_and_namerow(data, namerow):
  new_row = []
  first = get_value_of_tag(data, 'first')
  middle = get_value_of_tag(data, 'middle')
  last = get_value_of_tag(data, 'last')
  if(first in middle or middle in first):
    return None
  
  old_row = namerow[0]
  for i in range(4):
    if (i == 0):
      continue
    if "FIRSTNAME" not in old_row:
      namerow[i] = namerow[i].replace("FIRSTNAME", "")
    if "MIDDLENAME" not in old_row:
      namerow[i] = namerow[i].replace("MIDDLENAME", "")
    if "LASTNAME" not in old_row:
      namerow[i] = namerow[i].replace("LASTNAME", "")

  for element in namerow:
    new_row.append(element.replace("FIRSTNAME", first).replace("MIDDLENAME", middle).replace("LASTNAME", last))

  return new_row

In [ ]:
def generate_data_row():
  template = get_random_namerow()
  name_data = None
  while name_data == None: #Redo the step until generated place exists 
    name_data = grab_name_data()
  return combine_data_and_namerow(name_data, template)

In [ ]:
def generate_data(range_length, filename):
  with open('/drive/My Drive/Colab Notebooks/GeneratedData/' + filename, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(firstrow)
    print(range_length)
    for i in range(range_length):
      row = generate_data_row()
      if row == None:
        i = i - 1
        continue
      writer.writerow(row)

In [ ]:
# Generate all datasets
for element in data_length:
  filename = worksheetname + str(element) + '.csv'
  generate_data(element, filename)

10
50
100
250
500
1000
